In [1]:
import pandas as pd
import numpy as np

import json

import random

## Import Data

In [2]:
# # facebook data
# data_loc = "C:/Data/INCAS/Phase 2B Sample v2/sample_0805/sampled_facebook_en_tl_global_0805.jsonl"

# # reddit data
# data_loc = "C:/Data/INCAS/Phase 2B Sample v2/sample_0805/sampled_reddit_en_tl_global_0805.jsonl"

# # tumblr data
# data_loc = "C:/Data/INCAS/Phase 2B Sample v2/sample_0805/sampled_tumblr_en_tl_global_0805.jsonl"

# # twitter data
# data_loc = "C:/Data/INCAS/Phase 2B Sample v2/sample_0805/sampled_twitter_en_tl_global_0805.jsonl"

# challenge data 2
data_loc = "C:/Data/INCAS/challenge_problem_two_21NOV.jsonl/challenge_problem_two_21NOV.jsonl"

df = pd.read_json(path_or_buf=data_loc, lines=True)
df.columns

Index(['annotations', 'dataTags', 'embeddedUrls', 'extraAttributes',
       'imageUrls', 'segments', 'author', 'contentText', 'geolocation', 'id',
       'language', 'mediaType', 'mediaTypeAttributes', 'name', 'timePublished',
       'title', 'translatedContentText', 'translatedTitle', 'url'],
      dtype='object')

In [3]:
print(f"Number of rows: {len(df)}")

Number of rows: 3688879


In [ ]:
# df.to_pickle("phase2a_update_twitter.pkl")

In [ ]:
# df = pd.read_pickle("phase2a_update_twitter.pkl")

## Functions for Creating Data Quality Summary

In [4]:
def get_unique_value_summaries(df: pd.DataFrame, skip_columns=[]):

    unique_totals = []
    unique_percent = []

    for col in df.columns:
        if col in skip_columns:
            tot = np.nan
            perc = np.nan
        else:
            tot = len(df[col].value_counts())
            perc = tot/len(df)

        unique_totals.append(tot)
        unique_percent.append(perc)

    return unique_totals, unique_percent


def get_missing_summaries(df: pd.DataFrame):
    tot_miss = pd.DataFrame(df.isna().sum()).rename(columns={0:'Missing Values - Total'})

    perc_miss = pd.DataFrame(df.isna().sum()/len(df)).rename(columns={0:'Missing Values - %'})

    missing_summary = pd.concat([tot_miss, perc_miss], axis=1)

    return missing_summary


def get_examples(df: pd.DataFrame, num_examples=3):

    # get examples
    indices = df.index[:num_examples]
    examples = df.iloc[indices].transpose()

    # rename columns
    for col in examples.columns:
        examples.rename(columns={col:f"Example - row {col}"}, inplace=True)

    return examples


def get_data_type_summary(df: pd.DataFrame, num_rows: int):
    '''
    Given a dataframe, checks the type for the first number of
    rows in each column and returns the result. Handles the case
    where there are multiple types in a single column.

    To check all rows in the dataframe, set num_rows = len(df)

    Parameters
    ----------
    df: pd.DataFrame
        A dataframe containing data to check type of.
    
    num_rows: int
        The number of rows to consider when checking column
        value type. Will look at rows [0, num_rows).
    '''
    df.reset_index(inplace=True, drop=True)

    # dictionary to store data types
    df_col_types = {
        'attribute': [],
        'date_type': []
    }

    # get data types for each column
    for col in df.columns:

        # consider the first few rows of each column when determining type
        val_types = []
        for val in df.iloc[0:num_rows][col]:
            t = type(val)
            val_types.append(t)

        # get unique data types
        val_types = list(set(val_types))

        # clean the results
        val_types_clean = []
        for item in val_types:
            itemc = str(item).strip("<class ").rstrip(" >").replace("'", "")  # .strip()
            val_types_clean.append(itemc)
        val_types_clean.sort()

        # convert to string for easy reading
        if len(val_types_clean) > 1:
            val_types_str = ", ".join(val_types_clean)
        else:
            val_types_str = val_types_clean[0]

        df_col_types['attribute'].append(col)
        df_col_types['date_type'].append(val_types_str)

    type_summary = pd.DataFrame(df_col_types)
    type_summary.set_index('attribute', drop=True, inplace=True)

    return type_summary

### Generate Data Quality Summary

In [5]:
# prep columns containing lists
columns_convert = ["annotations", "dataTags", "embeddedUrls", "extraAttributes", "imageUrls", "segments"]
for col in columns_convert:
    entry_list = []
    for entry in df[col]:
        entry.sort()
        entry_str = ", ".join(entry)
        entry_list.append(entry_str)

    df[f"{col}_string"] = entry_list

In [6]:
df.columns

Index(['annotations', 'dataTags', 'embeddedUrls', 'extraAttributes',
       'imageUrls', 'segments', 'author', 'contentText', 'geolocation', 'id',
       'language', 'mediaType', 'mediaTypeAttributes', 'name', 'timePublished',
       'title', 'translatedContentText', 'translatedTitle', 'url',
       'annotations_string', 'dataTags_string', 'embeddedUrls_string',
       'extraAttributes_string', 'imageUrls_string', 'segments_string'],
      dtype='object')

In [8]:
df0 = df[['annotations_string', 'dataTags_string', 'embeddedUrls_string', 'extraAttributes_string',
          'imageUrls_string', 'segments_string', 'author', 'contentText', 'geolocation', 'id',
          'language', 'mediaType', 'mediaTypeAttributes', 
          'name', 'timePublished', 'title', 'url', 'translatedContentText',
          'translatedTitle']]  # 'mentionedUsers',

df1 = df[['annotations', 'dataTags', 'embeddedUrls', 'extraAttributes',
          'imageUrls', 'segments', 'author', 'contentText', 'geolocation', 'id',
          'language', 'mediaType', 'mediaTypeAttributes', 
          'name', 'timePublished', 'title', 'url', 'translatedContentText',
          'translatedTitle']]  # 'mentionedUsers',

unique_totals_list, unique_percent_list = get_unique_value_summaries(df0, skip_columns=['mediaTypeAttributes'])

missing_summary = get_missing_summaries(df1)

examples = get_examples(df1)

type_summary = get_data_type_summary(df1, len(df1))

summary_df = pd.concat([missing_summary, type_summary], axis=1)

summary_df['Uniqueness - Total'] = unique_totals_list
summary_df['Uniqueness - %'] = unique_percent_list

columns = list(summary_df.columns)
columns.sort()

summary_df = summary_df[columns]

summary_df = pd.concat([summary_df, examples], axis=1)
summary_df

,Missing Values - %,Missing Values - Total,Uniqueness - %,Uniqueness - Total,date_type,Example - row 0,Example - row 1,Example - row 2
annotations,0.000000,0,2.710851e-07,1.0,list,[],[],[]
dataTags,0.000000,0,2.710851e-07,1.0,list,[],[],[]
embeddedUrls,0.000000,0,2.459663e-02,90734.0,list,[],[https://alkoofiya.net/2023/08/31/israeli-forc...,[https://www.newarab.com/news/libyas-prime-min...
extraAttributes,0.000000,0,2.710851e-07,1.0,list,[],[],[]
imageUrls,0.000000,0,5.378599e-02,198410.0,list,[],[],[]
segments,0.000000,0,2.710851e-07,1.0,list,[],[],[]
author,0.971616,3584175,1.583083e-02,58398.0,"NoneType, str",waiv,None,None
contentText,0.000000,0,2.097374e-01,773696.0,str,The fact that they covered up Saudi Arab murde...,RT @Palnews22 Israeli forces detain 17 Palesti...,Libya's prime minister rejects normalization w...
geolocation,1.000000,3688879,0.000000e+00,0.0,float,NaN,NaN,NaN
id,0.000000,0,1.000000e+00,3688879.0,str,254b51c270d10b3de6b2863ec6a3568462e2af8a,b40146ee736fa722f06134005d1376d307b9cc66,1390241ea783b3ff66694dc1977ac4cc0a8ddf4e


In [9]:
summary_df.to_clipboard()